In [1]:
import torch
import os
import yaml
import argparse
from types import SimpleNamespace

def load_config(yaml_path):
    with open(yaml_path, 'r') as f:
        config = yaml.safe_load(f)
    return SimpleNamespace(**config)

def find_model_files(directory, dataset_name):
    """
    在指定目录下查找包含特定数据集名称的模型文件
    
    参数:
        directory (str): 要搜索的目录路径
        dataset_name (str): 要查找的数据集名称(如"ETTh1")
    
    返回:
        list: 匹配的模型文件路径列表
    """
    
    # 遍历目录下的所有文件和子目录
    for root, dirs, files in os.walk(directory):
        for dir in dirs:
            # 检查文件名中是否包含数据集名称(不区分大小写)
            if dataset_name.lower() in dir.lower():
                # 构建完整文件路径
                file_path = os.path.join(root, dir, "checkpoint.pth")
                yaml_path = os.path.join(root, dir, "config.yaml")
    
    return file_path, yaml_path

model_dir = "/scratch/sx2490/SOFTS_exp/PatchTST/PatchTST_supervised/checkpoints/solar_48_48_PatchTST_Solar_ftM_sl48_ll48_pl48_dm512_nh8_el2_dl1_df512_fc1_ebtimeF_dtTrue_Exp_0"
dataset_name = "solar"
model_path, yaml_path = find_model_files(model_dir, dataset_name)
config = load_config(yaml_path)
config.batch_size = 32

ModuleNotFoundError: No module named 'torch'

In [2]:
from exp.exp_long_term_forecasting import Exp_Long_Term_Forecast

exp = Exp_Long_Term_Forecast(config)
train_data, train_loader = exp._get_data(flag='train')
vali_data, vali_loader = exp._get_data(flag='val')
test_data, test_loader = exp._get_data(flag='test')
exp.model.load_state_dict(torch.load(model_path))
model = exp.model.eval()

Use GPU: cuda:0
train 36625
val 5137
test 10393


In [ ]:
import numpy as np
from tqdm import tqdm
import torch

mask_len = 48
criterion = torch.nn.MSELoss(reduction='none')
train_loss = []
train_loss_mask = []
count = 0
with torch.no_grad():
    for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in tqdm(enumerate(train_loader)):
        batch_x = batch_x.float().to(exp.device)
        batch_y = batch_y.float().to(exp.device)

        batch_x_mark = batch_x_mark.float().to(exp.device)
        batch_y_mark = batch_y_mark.float().to(exp.device)


        dec_inp = torch.zeros_like(batch_y[:, -exp.args.pred_len:, :]).float()
        dec_inp = torch.cat([batch_y[:, :exp.args.label_len, :], dec_inp], dim=1).float().to(exp.device)

        outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

        f_dim = -1 if exp.args.features == 'MS' else 0
        outputs = outputs[:, -exp.args.pred_len:, f_dim:]
        batch_y = batch_y[:, -exp.args.pred_len:, f_dim:].to(exp.device)
        loss = criterion(outputs, batch_y).mean(dim=[1, 2]).cpu()

        loss_mask_floats = []
        for i in range(mask_len):
            batch_x_mask = batch_x.float().to(exp.device)
            batch_x_mask[:, :i, :] = 0
            outputs_mask = model(batch_x_mask, batch_x_mark, dec_inp, batch_y_mark)
            loss_mask = criterion(outputs_mask, batch_y).mean(dim=[1, 2]).cpu()
            loss_mask_floats.append(loss_mask)

        torch.cuda.empty_cache()
        loss_mask_floats = torch.stack(loss_mask_floats).permute(1, 0)
        min_loss_mask_float = loss_mask_floats.min(-1).values
        diff = loss - min_loss_mask_float
        count += torch.sum(diff > 0).item()


        train_loss.extend(list(loss))
        train_loss_mask.extend(list(min_loss_mask_float))

np.average(train_loss), np.average(train_loss_mask), count / len(train_data)

1145it [02:29,  7.63it/s]


(0.18180756, 0.15542659, 5.387772925764192)

In [6]:
count/ len(train_data)

0.11797952218430034

In [4]:
import numpy as np
from tqdm import tqdm
import torch

mask_len = 48
criterion = torch.nn.MSELoss(reduction='none')
train_loss = []
train_loss_mask = []
count = 0
with torch.no_grad():
    for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in tqdm(enumerate(test_loader)):
        batch_x = batch_x.float().to(exp.device)
        batch_y = batch_y.float().to(exp.device)

        batch_x_mark = batch_x_mark.float().to(exp.device)
        batch_y_mark = batch_y_mark.float().to(exp.device)


        dec_inp = torch.zeros_like(batch_y[:, -exp.args.pred_len:, :]).float()
        dec_inp = torch.cat([batch_y[:, :exp.args.label_len, :], dec_inp], dim=1).float().to(exp.device)

        outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

        f_dim = -1 if exp.args.features == 'MS' else 0
        outputs = outputs[:, -exp.args.pred_len:, f_dim:]
        batch_y = batch_y[:, -exp.args.pred_len:, f_dim:].to(exp.device)
        loss = criterion(outputs, batch_y).mean(dim=[1, 2]).cpu()

        loss_mask_floats = []
        for i in range(mask_len):
            batch_x_mask = batch_x.float().to(exp.device)
            batch_x_mask[:, :i, :] = 0
            outputs_mask = model(batch_x_mask, batch_x_mark, dec_inp, batch_y_mark)
            loss_mask = criterion(outputs_mask, batch_y).mean(dim=[1, 2]).cpu()
            loss_mask_floats.append(loss_mask)

        torch.cuda.empty_cache()
        loss_mask_floats = torch.stack(loss_mask_floats).permute(1, 0)
        min_loss_mask_float = loss_mask_floats.min(-1).values
        diff = loss - min_loss_mask_float
        count += torch.sum(diff > 0).item()


        train_loss.extend(list(loss))
        train_loss_mask.extend(list(min_loss_mask_float))

np.average(train_loss), np.average(train_loss_mask), count / len(test_loader)

325it [00:42,  7.61it/s]


(0.27882376, 0.1679863, 13.295384615384615)

In [7]:
count/ len(test_data)

0.41576060810160687